In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNetCV
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning) # get rid of depreciation warnings

In [51]:
data = pd.read_csv("/Users/jacobbills/Desktop/Economics/Twitter_sentiment_DJIA30/Combined_stocks.csv") 
#brings in the csv

Let's make some percents for the different sentiment scores

In [52]:
data.info() #checking what we have

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14654 entries, 0 to 14653
Data columns (total 56 columns):
Unnamed: 0    14654 non-null int64
Date          14654 non-null object
HIGH          14654 non-null float64
CLOSE         14654 non-null float64
OPEN          14654 non-null float64
LOW           14654 non-null float64
NUM_NEG       14654 non-null int64
NUM_NEU       14654 non-null int64
NUM_POS       14654 non-null int64
TW            14654 non-null int64
weekend       14654 non-null int64
NUM_NEG1      14654 non-null int64
NUM_NEG2      14654 non-null int64
NUM_NEG3      14654 non-null int64
NUM_NEU1      14654 non-null int64
NUM_NEU2      14654 non-null int64
NUM_NEU3      14654 non-null int64
NUM_POS1      14654 non-null int64
NUM_POS2      14654 non-null int64
NUM_POS3      14654 non-null int64
TW1           14654 non-null int64
TW2           14654 non-null int64
TW3           14654 non-null int64
CLOSE1        14654 non-null float64
CLOSE2        14654 non-null float64
CLO

In [53]:
# create percentages
sentiments = ['NUM_NEG', 'NUM_NEG1', 'NUM_NEG2','NUM_NEU','NUM_NEU1','NUM_NEU2','NUM_POS','NUM_POS1','NUM_POS2']
for e in sentiments:
    if e.endswith("1"):
        data[e+"_PER"] = data[e]/data['TW1']
    elif e.endswith("2"):
        data[e+"_PER"] = data[e]/data['TW2']
    else:
        data[e+"_PER"] = data[e]/data['TW']

In [54]:
is_null = data.columns[data.isnull().any()] # seeing where there might be a problem
is_null

Index(['NUM_NEG_PER', 'NUM_NEG1_PER', 'NUM_NEG2_PER', 'NUM_NEU_PER',
       'NUM_NEU1_PER', 'NUM_NEU2_PER', 'NUM_POS_PER', 'NUM_POS1_PER',
       'NUM_POS2_PER'],
      dtype='object')

In [55]:
data[is_null].info() # about 30 missing for each, for whatever reason

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14654 entries, 0 to 14653
Data columns (total 9 columns):
NUM_NEG_PER     14620 non-null float64
NUM_NEG1_PER    14620 non-null float64
NUM_NEG2_PER    14619 non-null float64
NUM_NEU_PER     14620 non-null float64
NUM_NEU1_PER    14620 non-null float64
NUM_NEU2_PER    14619 non-null float64
NUM_POS_PER     14620 non-null float64
NUM_POS1_PER    14620 non-null float64
NUM_POS2_PER    14619 non-null float64
dtypes: float64(9)
memory usage: 1.0 MB


In [56]:
for e in is_null: #just fill them in with whatever is closest
    data[e].fillna(data[e].mean(), inplace=True)

In [57]:
for i in range(len(data)): # create the classes
    if data.loc[i,"CLOSE"] > data.loc[i,"OPEN"]+.25:
        data.loc[i,"class"] = 1 # gained by more than $.25
    elif data.loc[i,"CLOSE"] < data.loc[i,'OPEN']-.25:
        data.loc[i,"class"] = 2 # lost by more than $.25 
    else:
        data.loc[i,"class"] = 0 # minimal change

In [59]:
data[["CLOSE", "OPEN", "Date", "weekend","class"]] #see how the data looks

,CLOSE,OPEN,Date,weekend,class
0,76.02,76.50,6/4/13,0,2.0
1,74.79,76.04,6/5/13,0,2.0
2,76.24,74.75,6/6/13,0,1.0
3,78.03,76.68,6/7/13,0,1.0
4,78.03,76.68,6/8/13,1,1.0
5,78.03,76.68,6/9/13,1,1.0
6,78.27,77.98,6/10/13,0,1.0
7,76.52,77.61,6/11/13,0,2.0
8,74.72,76.91,6/12/13,0,2.0
9,75.21,74.46,6/13/13,0,1.0


In [60]:
data = data.loc[data.weekend==0]
data.head() #drop out the weekend data since it doesn't really tell us anything and see if it worked (it did)

,Unnamed: 0,Date,HIGH,CLOSE,OPEN,LOW,NUM_NEG,NUM_NEU,NUM_POS,TW,...,NUM_NEG_PER,NUM_NEG1_PER,NUM_NEG2_PER,NUM_NEU_PER,NUM_NEU1_PER,NUM_NEU2_PER,NUM_POS_PER,NUM_POS1_PER,NUM_POS2_PER,class
0,0,6/4/13,77.24,76.02,76.50,75.96,2,12,6,20,...,0.100000,0.000000,0.166667,0.600000,0.909091,0.833333,0.300000,0.090909,0.000000,2.0
1,1,6/5/13,76.28,74.79,76.04,74.64,0,4,2,6,...,0.000000,0.100000,0.000000,0.666667,0.600000,0.909091,0.333333,0.300000,0.090909,2.0
2,2,6/6/13,76.25,76.24,74.75,74.64,1,9,0,10,...,0.100000,0.000000,0.100000,0.900000,0.666667,0.600000,0.000000,0.333333,0.300000,1.0
3,3,6/7/13,78.12,78.03,76.68,76.45,2,4,11,17,...,0.117647,0.100000,0.000000,0.235294,0.900000,0.666667,0.647059,0.000000,0.333333,1.0
6,6,6/10/13,78.61,78.27,77.98,77.69,0,6,7,13,...,0.000000,0.000000,0.000000,0.461538,0.333333,0.500000,0.538462,0.666667,0.500000,1.0
7,7,6/11/13,77.85,76.52,77.61,76.50,2,9,8,19,...,0.105263,0.000000,0.000000,0.473684,0.461538,0.333333,0.421053,0.538462,0.666667,2.0
8,8,6/12/13,77.00,74.72,76.91,74.68,6,5,2,13,...,0.461538,0.105263,0.000000,0.384615,0.473684,0.461538,0.153846,0.421053,0.538462,2.0
9,9,6/13/13,75.26,75.21,74.46,73.79,2,21,2,25,...,0.080000,0.461538,0.105263,0.840000,0.384615,0.473684,0.080000,0.153846,0.421053,1.0
10,10,6/14/13,74.98,72.96,74.78,72.72,11,20,4,35,...,0.314286,0.080000,0.461538,0.571429,0.840000,0.384615,0.114286,0.080000,0.153846,2.0
13,13,6/17/13,74.21,73.86,73.69,73.39,2,14,0,16,...,0.125000,0.200000,0.000000,0.875000,0.200000,1.000000,0.000000,0.600000,0.000000,0.0


In [73]:
target = data.CLOSE # target
target_class = data["class"] # target for classifiers
high = data.HIGH # other columns that we don't really need for now
low = data.LOW 

In [74]:
data1 = data.drop(columns=['CLOSE', 'HIGH', 'LOW', 'Unnamed: 0', 'class']) # get rid of those ones

In [75]:
data_dummies = pd.get_dummies(data1) # make dummies for the dates

In [76]:
data_nodate = data1.drop(columns='Date') # create a df without the dates

In [77]:
data_prev = data_nodate.drop(columns=['NUM_NEG', 'NUM_NEU', 'NUM_POS', 'NUM_NEG_PER', 'NUM_NEU_PER', 'NUM_POS_PER', "TW"])
# create a df without anything for the day of interest

In [78]:
data_nopen = data_dummies.drop(columns='OPEN') # get rid of the target
data_nopen_date = data_nodate.drop(columns='OPEN')
data_nopen_prev = data_prev.drop(columns='OPEN') 

Now that the data is clean, we can try to run some different models on it

In [94]:
scaler = StandardScaler() #scale each of the three sets with open
scaler.fit(data_dummies)
scaled_dfo = scaler.transform(data_dummies)

scaler.fit(data_nodate)
scaled_df_noo = scaler.transform(data_nodate)

scaler.fit(data_prev)
scaled_prevo = scaler.transform(data_prev)

/Users/jacobbills/Desktop/python/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jacobbills/Desktop/python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jacobbills/Desktop/python/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jacobbills/Desktop/python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [95]:
# break into test and training sets
X_train, X_test, y_train, y_test = train_test_split(scaled_dfo, target, test_size=.2, random_state = 11)
X_train1, X_test1, y_train1, y_test1 = train_test_split(scaled_df_noo, target, test_size=.2, random_state = 11)
X_train2, X_test2, y_train2, y_test2 = train_test_split(scaled_prevo, target, test_size=.2, random_state = 11)

In [81]:
# try an elasting net and see how it works
clf = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .8, .9, .95, .99, 1], alphas= [.1, 1, 5, 10], cv=5) #basic Elastic Net with CV
clf.fit(X_train, y_train)
train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)
print("Train MSE: {}".format(mean_squared_error(y_train, train_predictions)))
print("Test MSE: {}".format(mean_squared_error(y_test, test_predictions)))

Train MSE: 0.6948907948158737
Test MSE: 0.6032102402205742


In [70]:
#plot_learning_curve(clf, "Learning Curve", X_train, y_train, cv=5)

In [82]:
# coefficient importances...really only the opening price matters
sorted(list(zip(data_dummies, clf.coef_)), key=lambda x: abs(x[1]))

[('NUM_NEG', -0.0),
 ('NUM_NEU', -0.0),
 ('NUM_POS', 0.0),
 ('TW', -0.0),
 ('weekend', 0.0),
 ('NUM_NEG1', -0.0),
 ('NUM_NEG2', 0.0),
 ('NUM_NEG3', 0.0),
 ('NUM_NEU1', -0.0),
 ('NUM_NEU2', -0.0),
 ('NUM_NEU3', -0.0),
 ('NUM_POS1', -0.0),
 ('NUM_POS2', 0.0),
 ('NUM_POS3', 0.0),
 ('TW1', -0.0),
 ('TW2', 0.0),
 ('TW3', 0.0),
 ('CLOSE2', 0.0),
 ('CLOSE3', 0.0),
 ('AXP', -0.0),
 ('BA', 0.0),
 ('CAT', 0.0),
 ('CSCO', -0.0),
 ('CVX', 0.0),
 ('DD', -0.0),
 ('DIS', -0.0),
 ('GE', -0.0),
 ('GS', 0.0),
 ('HD', -0.0),
 ('IBM', 0.0),
 ('INTC', -0.0),
 ('JNJ', 0.0),
 ('JPM', -0.0),
 ('KO', -0.0),
 ('MCD', 0.0),
 ('MMM', 0.0),
 ('MRK', -0.0),
 ('MSFT', -0.0),
 ('NKE', -0.0),
 ('PFE', -0.0),
 ('PG', -0.0),
 ('T', -0.0),
 ('TRV', 0.0),
 ('UNH', 0.0),
 ('UTX', 0.0),
 ('V', 0.0),
 ('VZ', -0.0),
 ('WMT', 0.0),
 ('XOM', 0.0),
 ('NUM_NEG1_PER', -0.0),
 ('NUM_NEG2_PER', 0.0),
 ('NUM_NEU_PER', -0.0),
 ('NUM_NEU1_PER', 0.0),
 ('NUM_NEU2_PER', -0.0),
 ('NUM_POS1_PER', -0.0),
 ('NUM_POS2_PER', 0.0),
 ('Date_1/10

In [83]:
# elastic net without dates
clf = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .8, .9, .95, .99, 1], alphas= [.1, 1, 5, 10], cv=5) #basic Elastic Net with CV
clf.fit(X_train1, y_train1)
train_predictions = clf.predict(X_train1)
test_predictions = clf.predict(X_test1)
print("Train MSE: {}".format(mean_squared_error(y_train1, train_predictions)))
print("Test MSE: {}".format(mean_squared_error(y_test1, test_predictions)))

Train MSE: 0.6948907948158737
Test MSE: 0.6032102402205742


In [84]:
# same as above
sorted(list(zip(data_nodate, clf.coef_)), key=lambda x: abs(x[1]))

[('NUM_NEG', -0.0),
 ('NUM_NEU', -0.0),
 ('NUM_POS', 0.0),
 ('TW', -0.0),
 ('weekend', 0.0),
 ('NUM_NEG1', -0.0),
 ('NUM_NEG2', 0.0),
 ('NUM_NEG3', 0.0),
 ('NUM_NEU1', -0.0),
 ('NUM_NEU2', -0.0),
 ('NUM_NEU3', -0.0),
 ('NUM_POS1', -0.0),
 ('NUM_POS2', 0.0),
 ('NUM_POS3', 0.0),
 ('TW1', -0.0),
 ('TW2', 0.0),
 ('TW3', 0.0),
 ('CLOSE2', 0.0),
 ('CLOSE3', 0.0),
 ('AXP', -0.0),
 ('BA', 0.0),
 ('CAT', 0.0),
 ('CSCO', -0.0),
 ('CVX', 0.0),
 ('DD', -0.0),
 ('DIS', -0.0),
 ('GE', -0.0),
 ('GS', 0.0),
 ('HD', -0.0),
 ('IBM', 0.0),
 ('INTC', -0.0),
 ('JNJ', 0.0),
 ('JPM', -0.0),
 ('KO', -0.0),
 ('MCD', 0.0),
 ('MMM', 0.0),
 ('MRK', -0.0),
 ('MSFT', -0.0),
 ('NKE', -0.0),
 ('PFE', -0.0),
 ('PG', -0.0),
 ('T', -0.0),
 ('TRV', 0.0),
 ('UNH', 0.0),
 ('UTX', 0.0),
 ('V', 0.0),
 ('VZ', -0.0),
 ('WMT', 0.0),
 ('XOM', 0.0),
 ('NUM_NEG1_PER', -0.0),
 ('NUM_NEG2_PER', 0.0),
 ('NUM_NEU_PER', -0.0),
 ('NUM_NEU1_PER', 0.0),
 ('NUM_NEU2_PER', -0.0),
 ('NUM_POS1_PER', -0.0),
 ('NUM_POS2_PER', 0.0),
 ('CLOSE1', 

In [85]:
# elastic net without today's data
clf = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .8, .9, .95, .99, 1], alphas= [.1, 1, 5, 10], cv=5) #basic Elastic Net with CV
clf.fit(X_train2, y_train2)
train_predictions = clf.predict(X_train2)
test_predictions = clf.predict(X_test2)
print("Train MSE: {}".format(mean_squared_error(y_train2, train_predictions)))
print("Test MSE: {}".format(mean_squared_error(y_test2, test_predictions)))

Train MSE: 0.7003492633051123
Test MSE: 0.6075214545452414


In [86]:
sorted(list(zip(data_prev, clf.coef_)), key=lambda x: abs(x[1]))

[('weekend', 0.0),
 ('NUM_NEG1', -0.0),
 ('NUM_NEG2', 0.0),
 ('NUM_NEG3', 0.0),
 ('NUM_NEU1', -0.0),
 ('NUM_NEU2', -0.0),
 ('NUM_NEU3', -0.0),
 ('NUM_POS1', -0.0),
 ('NUM_POS2', 0.0),
 ('NUM_POS3', 0.0),
 ('TW1', -0.0),
 ('TW2', 0.0),
 ('TW3', 0.0),
 ('CLOSE2', 0.0),
 ('CLOSE3', 0.0),
 ('AXP', -0.0),
 ('BA', 0.0),
 ('CAT', 0.0),
 ('CSCO', -0.0),
 ('CVX', 0.0),
 ('DD', -0.0),
 ('DIS', -0.0),
 ('GE', -0.0),
 ('GS', 0.0),
 ('HD', -0.0),
 ('IBM', 0.0),
 ('INTC', -0.0),
 ('JNJ', 0.0),
 ('JPM', -0.0),
 ('KO', -0.0),
 ('MCD', 0.0),
 ('MMM', 0.0),
 ('MRK', -0.0),
 ('MSFT', -0.0),
 ('NKE', -0.0),
 ('PFE', -0.0),
 ('PG', -0.0),
 ('T', -0.0),
 ('TRV', 0.0),
 ('UNH', 0.0),
 ('UTX', 0.0),
 ('V', 0.0),
 ('VZ', -0.0),
 ('WMT', 0.0),
 ('XOM', 0.0),
 ('NUM_NEG1_PER', -0.0),
 ('NUM_NEG2_PER', 0.0),
 ('NUM_NEU1_PER', 0.0),
 ('NUM_NEU2_PER', -0.0),
 ('NUM_POS1_PER', -0.0),
 ('NUM_POS2_PER', 0.0),
 ('CLOSE1', 0.0004528626066566547),
 ('OPEN', 45.02575353669932)]

In [96]:
# scale the data wihtout open
scaler = StandardScaler()
scaler.fit(data_nopen)
scaled_dfn = scaler.transform(data_nopen)

scaler.fit(data_nopen_date)
scaled_df_non = scaler.transform(data_nopen_date)

scaler.fit(data_nopen_prev)
scaled_prevn = scaler.transform(data_nopen_prev)

/Users/jacobbills/Desktop/python/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jacobbills/Desktop/python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jacobbills/Desktop/python/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jacobbills/Desktop/python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [97]:
# create training sets
X_train3, X_test3, y_train3, y_test3 = train_test_split(scaled_dfn, target, test_size=.2, random_state = 11)
X_train4, X_test4, y_train4, y_test4 = train_test_split(scaled_df_non, target, test_size=.2, random_state = 11)
X_train5, X_test5, y_train5, y_test5 = train_test_split(scaled_prevn, target, test_size=.2, random_state = 11)

In [89]:
# Elastic net with dates
clf = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .8, .9, .95, .99, 1], alphas= [.1, 1, 5, 10], cv=5) #basic Elastic Net with CV
clf.fit(X_train3, y_train3)
train_predictions = clf.predict(X_train3)
test_predictions = clf.predict(X_test3)
print("Train MSE: {}".format(mean_squared_error(y_train3, train_predictions)))
print("Test MSE: {}".format(mean_squared_error(y_test3, test_predictions)))
print(sorted(list(zip(data_nopen, clf.coef_)), key=lambda x: abs(x[1])))

Train MSE: 0.9560548986951125
Test MSE: 0.7876493577160306
[('NUM_NEG', -0.0), ('NUM_NEU', -0.0), ('NUM_POS', 0.0), ('TW', -0.0), ('weekend', 0.0), ('NUM_NEG1', -0.0), ('NUM_NEG2', 0.0), ('NUM_NEG3', 0.0), ('NUM_NEU1', -0.0), ('NUM_NEU2', -0.0), ('NUM_NEU3', -0.0), ('NUM_POS1', -0.0), ('NUM_POS2', -0.0), ('NUM_POS3', 0.0), ('TW1', -0.0), ('TW2', -0.0), ('TW3', -0.0), ('AXP', -0.0), ('BA', 0.0), ('CAT', 0.0), ('CSCO', -0.0), ('CVX', 0.0), ('DD', -0.0), ('DIS', -0.0), ('GE', -0.0), ('GS', 0.0), ('HD', -0.0), ('IBM', 0.0), ('INTC', -0.0), ('JNJ', 0.0), ('JPM', 0.0), ('KO', -0.0), ('MCD', 0.0), ('MMM', 0.0), ('MRK', -0.0), ('MSFT', -0.0), ('NKE', -0.0), ('PFE', -0.0), ('PG', -0.0), ('T', -0.0), ('TRV', 0.0), ('UNH', -0.0), ('UTX', 0.0), ('V', 0.0), ('VZ', -0.0), ('WMT', 0.0), ('XOM', 0.0), ('NUM_NEG1_PER', -0.0), ('NUM_NEG2_PER', 0.0), ('NUM_NEU_PER', -0.0), ('NUM_NEU1_PER', 0.0), ('NUM_NEU2_PER', -0.0), ('NUM_POS1_PER', -0.0), ('NUM_POS2_PER', 0.0), ('Date_1/10/14', -0.0), ('Date_1/13/14'

In [90]:
# Elastic net with no dates
clf = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .8, .9, .95, .99, 1], alphas= [.1, 1, 5, 10], cv=5) #basic Elastic Net with CV
clf.fit(X_train4, y_train4)
train_predictions = clf.predict(X_train4)
test_predictions = clf.predict(X_test4)
print("Train MSE: {}".format(mean_squared_error(y_train4, train_predictions)))
print("Test MSE: {}".format(mean_squared_error(y_test4, test_predictions)))
print(sorted(list(zip(data_nopen_date, clf.coef_)), key=lambda x: abs(x[1])))

Train MSE: 0.9595441609186985
Test MSE: 0.7892444502058501
[('NUM_NEG', -0.0), ('NUM_NEU', -0.0), ('NUM_POS', 0.0), ('TW', -0.0), ('weekend', 0.0), ('NUM_NEG1', -0.0), ('NUM_NEG2', 0.0), ('NUM_NEG3', 0.0), ('NUM_NEU1', -0.0), ('NUM_NEU2', -0.0), ('NUM_NEU3', -0.0), ('NUM_POS1', -0.0), ('NUM_POS2', -0.0), ('NUM_POS3', 0.0), ('TW1', -0.0), ('TW2', -0.0), ('TW3', -0.0), ('AXP', -0.0), ('BA', 0.0), ('CAT', 0.0), ('CSCO', -0.0), ('CVX', 0.0), ('DD', -0.0), ('DIS', -0.0), ('GE', -0.0), ('GS', 0.0), ('HD', -0.0), ('IBM', 0.0), ('INTC', -0.0), ('JNJ', 0.0), ('JPM', 0.0), ('KO', -0.0), ('MCD', 0.0), ('MMM', 0.0), ('MRK', -0.0), ('MSFT', -0.0), ('NKE', -0.0), ('PFE', -0.0), ('PG', -0.0), ('T', -0.0), ('TRV', 0.0), ('UNH', -0.0), ('UTX', 0.0), ('V', 0.0), ('VZ', -0.0), ('WMT', 0.0), ('XOM', 0.0), ('NUM_NEG1_PER', -0.0), ('NUM_NEG2_PER', 0.0), ('NUM_NEU_PER', -0.0), ('NUM_NEU1_PER', 0.0), ('NUM_NEU2_PER', -0.0), ('NUM_POS1_PER', -0.0), ('NUM_POS2_PER', 0.0), ('CLOSE3', 0.0011064105615996274), ('CL

In [91]:
# elastic net without anything from the day being predicted
clf = ElasticNetCV(l1_ratio=[.1, .3, .5, .7, .8, .9, .95, .99, 1], alphas= [.1, 1, 5, 10], cv=5) #basic Elastic Net with CV
clf.fit(X_train5, y_train5)
train_predictions = clf.predict(X_train5)
test_predictions = clf.predict(X_test5)
print("Train MSE: {}".format(mean_squared_error(y_train5, train_predictions)))
print("Test MSE: {}".format(mean_squared_error(y_test5, test_predictions)))
print(sorted(list(zip(data_nopen_prev, clf.coef_)), key=lambda x: abs(x[1])))

Train MSE: 1.007451900843909
Test MSE: 0.8312475708031988
[('weekend', 0.0), ('NUM_NEG1', -0.0), ('NUM_NEG2', -0.0), ('NUM_NEG3', 0.0), ('NUM_NEU1', -0.0), ('NUM_NEU2', -0.0), ('NUM_NEU3', -0.0), ('NUM_POS1', -0.0), ('NUM_POS2', -0.0), ('NUM_POS3', 0.0), ('TW1', -0.0), ('TW2', -0.0), ('TW3', -0.0), ('CLOSE2', 0.0), ('AXP', -0.0), ('BA', 0.0), ('CAT', 0.0), ('CSCO', -0.0), ('CVX', 0.0), ('DD', -0.0), ('DIS', 0.0), ('GE', -0.0), ('GS', 0.0), ('HD', -0.0), ('IBM', 0.0), ('INTC', -0.0), ('JNJ', 0.0), ('JPM', -0.0), ('KO', -0.0), ('MCD', -0.0), ('MMM', 0.0), ('MRK', -0.0), ('MSFT', -0.0), ('NKE', 0.0), ('PFE', -0.0), ('PG', -0.0), ('T', -0.0), ('TRV', 0.0), ('UNH', 0.0), ('UTX', 0.0), ('V', 0.0), ('VZ', -0.0), ('WMT', -0.0), ('XOM', 0.0), ('NUM_NEG1_PER', -0.0), ('NUM_NEG2_PER', 0.0), ('NUM_NEU1_PER', 0.0), ('NUM_NEU2_PER', -0.0), ('NUM_POS1_PER', 0.0), ('NUM_POS2_PER', 0.0), ('CLOSE3', 0.0004698468656508827), ('CLOSE1', 45.012002825723954)]


As can be seen, it isn't very interesting to use the data to predict future prices because it ends up that basically the only thing you want to know is what the opening price/yesterday's price is, and then how people were tweeting the day of (possibly after closing). So instead we'll try to predict if tweets can forecast if the stock is going to go up or down. The "previous" data is the most important sort here

In [100]:
from sklearn.neighbors import KNeighborsClassifier #Nearest Neighbors
from sklearn.naive_bayes import MultinomialNB, ComplementNB, GaussianNB # different Bayesian models
from sklearn.svm import SVC #support vector machine
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier #ensemble models

from sklearn.model_selection import GridSearchCV #to do cross validation

# Metrics
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error #different ways of scoring
from sklearn.metrics import classification_report # classification report
from sklearn.metrics import confusion_matrix # confusion matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc

In [101]:
# create training sets from the previous day data (with and without OPEN)
X_traincO, X_testcO, y_traincO, y_testcO = train_test_split(scaled_prevo, target_class, test_size=.2, random_state = 11)
X_trainc, X_testc, y_trainc, y_testc = train_test_split(scaled_prevn, target_class, test_size=.2, random_state = 11)

In [103]:
# K Nearest Neighbors with open
knn = KNeighborsClassifier()
gridsearch = GridSearchCV(knn, {"n_neighbors": [5, 7, 9, 11], "weights": ['uniform', 'distance'], 
                                'p': [1, 2, 3]}, scoring='f1_micro', cv=3)
gridsearch.fit(X_traincO, y_traincO)
print("Best Params: {}".format(gridsearch.best_params_))
y_pred_train = gridsearch.predict(X_traincO)
print("Train F1: {}".format(f1_score(y_traincO, y_pred_train, average=None)))
print("Test Classification Report:")
y_pred_test = gridsearch.predict(X_testcO)
print(classification_report(y_testcO, y_pred_test))
print("Train Accuracy: {}\tTest accuracy: {}".format(accuracy_score(y_traincO, y_pred_train),
                                                     accuracy_score(y_testcO, y_pred_test)))

Best Params: {'n_neighbors': 11, 'p': 3, 'weights': 'distance'}
Train F1: [1. 1. 1.]
Test Classification Report:
              precision    recall  f1-score   support

         0.0       0.55      0.65      0.59       810
         1.0       0.42      0.43      0.43       630
         2.0       0.39      0.29      0.33       592

   micro avg       0.47      0.47      0.47      2032
   macro avg       0.45      0.45      0.45      2032
weighted avg       0.46      0.47      0.46      2032

Train Accuracy: 1.0	Test accuracy: 0.4749015748031496


In [105]:
# confusion matric to evaluate performance
confusion_matrix(y_testcO, y_pred_test)

array([[525, 166, 119],
       [215, 271, 144],
       [222, 201, 169]])

In [106]:
# K Nearest Neighbors without open
knn = KNeighborsClassifier()
gridsearch = GridSearchCV(knn, {"n_neighbors": [5, 7, 9, 11], "weights": ['uniform', 'distance'], 
                                'p': [1, 2, 3]}, scoring='f1_micro', cv=3)
gridsearch.fit(X_trainc, y_trainc)
print("Best Params: {}".format(gridsearch.best_params_))
y_pred_train = gridsearch.predict(X_trainc)
print("Train F1: {}".format(f1_score(y_trainc, y_pred_train, average=None)))
print("Test Classification Report:")
y_pred_test = gridsearch.predict(X_testc)
print(classification_report(y_testc, y_pred_test))
print("Train Accuracy: {}\tTest accuracy: {}".format(accuracy_score(y_trainc, y_pred_train),
                                                     accuracy_score(y_testc, y_pred_test)))
print(confusion_matrix(y_testc, y_pred_test))

Best Params: {'n_neighbors': 11, 'p': 3, 'weights': 'distance'}
Train F1: [1. 1. 1.]
Test Classification Report:
              precision    recall  f1-score   support

         0.0       0.55      0.65      0.59       810
         1.0       0.43      0.43      0.43       630
         2.0       0.39      0.29      0.33       592

   micro avg       0.48      0.48      0.48      2032
   macro avg       0.46      0.46      0.45      2032
weighted avg       0.46      0.48      0.47      2032

Train Accuracy: 1.0	Test accuracy: 0.4763779527559055
[[525 165 120]
 [212 272 146]
 [220 201 171]]


So first of all this model is way overfitting, so that needs to be fixed. Removing "OPEN" made it only marginally better (talking like 2 more true positives for each class) so that indicates that OPEN is not a driving force for this problem (unlike the basic prediction problem). Secondly, it seems to mostly put things in the first class (no major change) which works okay but not great. It is really bad at predicting a drop in prices, so I'm not sure what is going on there, except that it probaly trained itself to mostly put things in the first class. I'll try a random forest, since we can get feature importances from that which may give valuable information about what variables to keep and what ones to drop so that we stop overfitting the nearest neighbors. 

In [ ]:
# RandomForestClassifier with Open
rfc = RandomForestClassifier()
gridsearch = GridSearchCV(rfc, {"n_estimators": [20, 50, 100, 200, 500], "max_depth": [1, 3, 5, 7, 9, 11], 
                                 "min_samples_leaf": [3 , 5 , 7 , 9], "max_features": [None, 'sqrt', 'log2']
                                , "class_weight": ["balanced", "balanced_subsample"], 'random_state': [9]}, 
                          scoring='f1_micro', cv=3)
gridsearch.fit(X_traincO, y_traincO)
print("Best Params: {}".format(gridsearch.best_params_))
y_pred_train = gridsearch.predict(X_traincO)
print("Train F1: {}".format(f1_score(y_traincO, y_pred_train, average=None)))
print("Test Classification Report:")
y_pred_test = gridsearch.predict(X_testcO)
print(classification_report(y_testcO, y_pred_test))
print("Train Accuracy: {}\tTest accuracy: {}".format(accuracy_score(y_traincO, y_pred_train),
                                                     accuracy_score(y_testcO, y_pred_test)))
print(confusion_matrix(y_testcO, y_pred_test))

In [ ]:
rfcb = RandomForestClassifier()
feature_imp_rfc1 = sorted(list(zip(data_prev, rfcb.feature_importances_)), key=lambda x: x[1], reverse=True)

In [ ]:
# RandomForestClassifier without Open
gridsearch = GridSearchCV(rfc, {"n_estimators": [20, 50, 100, 200, 500], "max_depth": [1, 3, 5, 7, 9, 11], 
                                 "min_samples_leaf": [3 , 5 , 7 , 9], "max_features": [None, 'sqrt', 'log2']
                                , "class_weight": ["balanced", "balanced_subsample"], 'random_state': [9]}, 
                          scoring='f1_micro', cv=3)
gridsearch.fit(X_trainc, y_trainc)
print("Best Params: {}".format(gridsearch.best_params_))
y_pred_train = gridsearch.predict(X_trainc)
print("Train F1: {}".format(f1_score(y_trainc, y_pred_train, average=None)))
print("Test Classification Report:")
y_pred_test = gridsearch.predict(X_testc)
print(classification_report(y_testc, y_pred_test))
print("Train Accuracy: {}\tTest accuracy: {}".format(accuracy_score(y_trainc, y_pred_train),
                                                     accuracy_score(y_testc, y_pred_test)))
print(confusion_matrix(y_testc, y_pred_test))

In [ ]:
rfcb2 = RandomForestClassifier()
feature_imp_rfc2 = sorted(list(zip(data_nopen_prev, rfcb2.feature_importances_)), key=lambda x: x[1], reverse=True)

I'll now use an old set of functions I wrote to get plot out the most important features and then decide how to engineer from there.

In [ ]:
def key_extract(dic):
    y = pd.DataFrame.from_dict(dic, orient='index') #turns the dictionary into a data frame
    y = y.index # list of the names in the dataframe
    return y

def feature_extract(data_frame, feature_importances, sensitivity): # function to extract a list of features above a given sensitivty
    coef_dict = dict((zip(data_frame, feature_importances))) #creates a dictionary from the model's feature importance
    coef_list = {} # another dictionary
    for key in coef_dict: # runs through the zipped dictionary 
        x = coef_dict.get(key) # extracts value for the key
        if abs(x) > sensitivity:  # if the absolute value is bigger than the sensitivity
            coef_list[key] = x # adds a new entry to the other dictionary 
        else: # if the coefficient isn't big enough
            pass # move to the next key
    coef_list = key_extract(coef_list) # uses a previous function I wrote to make a list of the names of those coeff
    return coef_list # returns that list

def best_feature_dict(list_, features_imp): #creates a dictionary from a list of tuples using another list of names
    y = {}
    for e in list_:
        y[e] = dict(features_imp).get(e)
    return y

def best_feature_plot(list_, features_imp, ticks=90): #plots the features importances
    y = best_feature_dict(list_, features_imp)
    plt.bar(y.keys(),y.values())
    plt.title("Feature Importance")
    plt.xticks(rotation=ticks)
    plt.ylabel("Percent Used")
    plt.grid(True)
    
def features_full(data_frame, feature_importances, sensitivity): #combines these functions into one nice one
    x = feature_extract(data_frame, feature_importances, sensitivity)
    y = best_feature_plot(x, features_imp)
    return x, y